# 梧桐杯 AI 安全告警智能研判系统 - Colab 训练

## 📋 训练流程

1. ✅ 克隆 GitHub 仓库
2. ✅ 安装依赖
3. ✅ 检查数据
4. ✅ 数据预处理
5. ✅ 训练 V2 模型（XGBoost + CatBoost + LightGBM）
6. ✅ 查看训练结果
7. ✅ 下载模型文件

## ⚙️ 运行前准备

1. 确保运行时类型设置为 **GPU**（Runtime > Change runtime type > GPU）
2. 按顺序执行每个单元格
3. 训练时间约 10-15 分钟

---

In [1]:
# 1. 克隆 GitHub 仓库
!git clone https://github.com/alltobebetter/WuTong.git
%cd WuTong

# 查看项目结构
!ls -la

Cloning into 'WuTong'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 74 (delta 14), reused 74 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 1.56 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/WuTong
total 104
drwxr-xr-x 8 root root  4096 Feb 14 14:23 .
drwxr-xr-x 1 root root  4096 Feb 14 14:23 ..
-rw-r--r-- 1 root root  2481 Feb 14 14:23 CHANGELOG.md
-rw-r--r-- 1 root root  4266 Feb 14 14:23 COLAB_GUIDE.md
-rw-r--r-- 1 root root 13448 Feb 14 14:23 colab_train.ipynb
drwxr-xr-x 4 root root  4096 Feb 14 14:23 data
drwxr-xr-x 8 root root  4096 Feb 14 14:23 .git
-rw-r--r-- 1 root root   244 Feb 14 14:23 .gitignore
drwxr-xr-x 2 root root  4096 Feb 14 14:23 legacy
drwxr-xr-x 2 root root  4096 Feb 14 14:23 models
-rw-r--r-- 1 root root  4689 Feb 14 14:23 PROJECT_PLAN.txt
-rw-r--r-- 1 root root  3003 Feb 14 14:23 QUICK_START.md
-rw-r--r-- 1 root 

In [2]:
# 2. 安装依赖
print("📦 安装依赖包...")
!pip install -q -r requirements-colab.txt

print("\n✅ 依赖安装完成！")

# 验证关键包
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import pandas as pd

print(f"XGBoost 版本: {xgb.__version__}")
print(f"CatBoost 版本: {cb.__version__}")
print(f"LightGBM 版本: {lgb.__version__}")
print(f"Pandas 版本: {pd.__version__}")

📦 安装依赖包...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.6 MB/s eta 0:00:00

✅ 依赖安装完成！
XGBoost 版本: 3.1.3
CatBoost 版本: 1.2.8
LightGBM 版本: 4.6.0
Pandas 版本: 2.2.2


In [3]:
# 3. 检查 GPU 可用性
import torch

if torch.cuda.is_available():
    print(f"✅ GPU 可用: {torch.cuda.get_device_name(0)}")
    print(f"   显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ GPU 不可用，将使用 CPU 训练（速度较慢）")
    print("   建议: Runtime > Change runtime type > GPU")

✅ GPU 可用: Tesla T4
   显存: 14.56 GB


In [4]:
# 4. 检查数据集
import pandas as pd
from pathlib import Path

# 查找数据文件
data_files = list(Path('data').rglob('*.xlsx'))
print(f"找到 {len(data_files)} 个数据文件:")
for f in data_files:
    print(f"  - {f}")

if data_files:
    # 读取第一个数据文件
    df = pd.read_excel(data_files[0])
    print(f"\n数据集大小: {len(df)} 条")
    print(f"\n列名: {df.columns.tolist()}")
    print(f"\n攻击类型分布:")
    print(df.iloc[:, -1].value_counts())
else:
    print("\n⚠️ 未找到数据文件！")
    print("请确保 data/ 目录下有数据文件")

找到 2 个数据文件:
  - data/原始告警信息样例数据集V1.2.xlsx
  - data/raw/原始告警信息样例数据集V1.2.xlsx

数据集大小: 11000 条

列名: ['timestamp', 'user_agent', 'method', 'src_ip', 'dst_ip', 'url_path', 'request_body', 'attack_type(结果列，需参赛选手判断输出)']

攻击类型分布:
attack_type(结果列，需参赛选手判断输出)
正常访问              3413
目录遍历攻击             972
远程命令执行攻击           965
SQL注入攻击            964
文件包含攻击             959
XSS跨站脚本攻击          958
CSRF攻击             942
Java反序列化漏洞利用攻击     920
文件上传攻击             907
Name: count, dtype: int64


In [5]:
# 5. 数据预处理
print("🔄 开始数据预处理...")

# 查找 Excel 文件
import glob
excel_files = glob.glob('data/**/*.xlsx', recursive=True)

if excel_files:
    data_file = excel_files[0]
    print(f"使用数据文件: {data_file}")

    # 运行预处理
    !python scripts/ingest.py "{data_file}"

    print("\n✅ 数据预处理完成！")

    # 检查生成的 parquet 文件
    parquet_files = glob.glob('data/staging/*.parquet')
    print(f"\n生成的 parquet 文件: {parquet_files}")
else:
    print("❌ 未找到 Excel 数据文件！")

🔄 开始数据预处理...
使用数据文件: data/原始告警信息样例数据集V1.2.xlsx
2026-02-14 14:24:01,353 INFO 读取 原始告警信息样例数据集V1.2.xlsx 成功，形状: (11000, 8)
2026-02-14 14:24:01,456 INFO 已保存标准化数据: /content/WuTong/data/staging/原始告警信息样例数据集V1.2.parquet  (11000 条)
✔ 已生成: /content/WuTong/data/staging/原始告警信息样例数据集V1.2.parquet

✅ 数据预处理完成！

生成的 parquet 文件: ['data/staging/原始告警信息样例数据集V1.2.parquet']


In [6]:
# 6. 训练 V2 模型（完整训练，带交叉验证）
print("🚀 开始训练 V2 模型...")
print("模型: XGBoost + CatBoost + LightGBM")
print("交叉验证: 5 折")
print("预计时间: 10-15 分钟\n")

!python scripts/train_v2.py --version v2.0.0

print("\n✅ 训练完成！")

🚀 开始训练 V2 模型...
模型: XGBoost + CatBoost + LightGBM
交叉验证: 5 折
预计时间: 10-15 分钟

使用数据: /content/WuTong/data/staging/原始告警信息样例数据集V1.2.parquet
模型版本: v2.0.0
交叉验证: 是 (5 折)
2026-02-14 14:24:08,078 INFO === 训练开始 V2: version=v2.0.0 ===
2026-02-14 14:24:08,101 INFO 数据加载: 11000 条
2026-02-14 14:24:08,103 INFO 攻击类型分布:
attack_type
正常访问              3413
目录遍历攻击             972
远程命令执行攻击           965
SQL注入攻击            964
文件包含攻击             959
XSS跨站脚本攻击          958
CSRF攻击             942
Java反序列化漏洞利用攻击     920
文件上传攻击             907
Name: count, dtype: int64
2026-02-14 14:24:08,104 INFO 特征提取 ...
2026-02-14 14:24:08,104 INFO 开始特征提取，共 11000 条记录 ...
2026-02-14 14:24:08,540 INFO   特征提取进度: 2000 / 11000
2026-02-14 14:24:09,114 INFO   特征提取进度: 4000 / 11000
2026-02-14 14:24:09,854 INFO   特征提取进度: 6000 / 11000
2026-02-14 14:24:10,626 INFO   特征提取进度: 8000 / 11000
2026-02-14 14:24:11,242 INFO   特征提取进度: 10000 / 11000
2026-02-14 14:24:11,545 INFO 特征提取完成，新增 26 个特征列
2026-02-14 14:24:11,551 INFO 训练告警聚合器 ...
2026-02-14 14

In [7]:
# 6-alternative. 快速训练（不带交叉验证，约 5 分钟）
# 如果想快速测试，可以运行这个单元格代替上面的单元格

# print("⚡ 快速训练模式（不带交叉验证）...")
# !python scripts/train_v2.py --version v2.0.0-fast --no-cv
# print("\n✅ 快速训练完成！")

In [8]:
# 7. 查看训练结果
import json
from pathlib import Path

manifest_path = Path('models/v2.0.0/manifest.json')

if manifest_path.exists():
    with open(manifest_path, 'r', encoding='utf-8') as f:
        manifest = json.load(f)

    print("=" * 60)
    print("📊 训练结果")
    print("=" * 60)
    print(f"\n版本: {manifest['version']}")
    print(f"训练时间: {manifest['trained_at']}")
    print(f"数据量: {manifest['data_rows']} 条")
    print(f"特征数: {len(manifest['feature_list'])} 个")
    print(f"类别数: {len(manifest['classes'])} 类")

    print("\n" + "=" * 60)
    print("🎯 模型性能")
    print("=" * 60)

    metrics = manifest['metrics']

    # 单模型性能
    for model_name in ['xgboost', 'catboost', 'lightgbm']:
        if model_name in metrics:
            m = metrics[model_name]
            print(f"\n{model_name.upper()}:")
            print(f"  测试准确率: {m['test_accuracy']:.4f}")
            print(f"  测试 F1: {m['test_f1']:.4f}")
            if m.get('cv_accuracy'):
                print(f"  交叉验证准确率: {m['cv_accuracy']:.4f}")

    # 集成模型性能
    if 'ensemble' in metrics:
        e = metrics['ensemble']
        print(f"\n{'='*60}")
        print(f"🏆 集成模型（最终模型）")
        print(f"{'='*60}")
        print(f"  准确率: {e['test_accuracy']:.4f} ({e['test_accuracy']*100:.2f}%)")
        print(f"  F1 分数: {e['test_f1']:.4f}")

    print("\n" + "=" * 60)

    # 读取详细分类报告
    report_path = Path('models/v2.0.0/classification_report.txt')
    if report_path.exists():
        print("\n📋 详细分类报告:")
        print("=" * 60)
        with open(report_path, 'r', encoding='utf-8') as f:
            print(f.read())
else:
    print("❌ 未找到训练结果文件！")
    print("请先运行训练单元格")

📊 训练结果

版本: v2.0.0
训练时间: 2026-02-14T14:25:16.493378+00:00
数据量: 11000 条
特征数: 25 个
类别数: 9 类

🎯 模型性能

XGBOOST:
  测试准确率: 0.9768
  测试 F1: 0.9768
  交叉验证准确率: 0.9710

CATBOOST:
  测试准确率: 0.9750
  测试 F1: 0.9751
  交叉验证准确率: 0.9690

LIGHTGBM:
  测试准确率: 0.9750
  测试 F1: 0.9750
  交叉验证准确率: 0.9706

🏆 集成模型（最终模型）
  准确率: 0.9759 (97.59%)
  F1 分数: 0.9760


📋 详细分类报告:
                precision    recall  f1-score   support

        CSRF攻击     0.8974    0.9309    0.9138       188
Java反序列化漏洞利用攻击     1.0000    1.0000    1.0000       184
       SQL注入攻击     0.9787    0.9534    0.9659       193
     XSS跨站脚本攻击     1.0000    0.9896    0.9948       192
        文件上传攻击     0.9945    1.0000    0.9972       181
        文件包含攻击     0.9784    0.9427    0.9602       192
          正常访问     0.9666    0.9751    0.9708       683
        目录遍历攻击     0.9948    0.9948    0.9948       194
      远程命令执行攻击     1.0000    1.0000    1.0000       193

      accuracy                         0.9759      2200
     macro avg     0.9789    0.9763  

In [9]:
# 8. 对比不同版本模型（如果有 V1 模型）
!python scripts/compare_models.py

找到 1 个模型版本: v2.0.0

+-------------+---------------------+----------+-------------+----------+----------+
| 版本        | 训练时间            | 数据量   | 模型类型    |   准确率 |   F1分数 |
+=============+=====================+==========+=============+==========+==========+
| v2.0.0      | 2026-02-14T14:25:16 | 11000    | XGB+Cat+LGB |   0.9759 |   0.976  |
+-------------+---------------------+----------+-------------+----------+----------+
| └─ XGBoost  |                     |          |             |   0.9768 |   0.9768 |
+-------------+---------------------+----------+-------------+----------+----------+
| └─ CatBoost |                     |          |             |   0.975  |   0.9751 |
+-------------+---------------------+----------+-------------+----------+----------+
| └─ LightGBM |                     |          |             |   0.975  |   0.975  |
+-------------+---------------------+----------+-------------+----------+----------+

=== 详细信息 ===

【v2.0.0】
  训练时间: 2026-02-14T14:25:16.493378+00:0

In [10]:
# 9. 打包模型文件准备下载
print("📦 打包模型文件...")

!zip -r models_v2.0.0.zip models/v2.0.0/

print("\n✅ 打包完成！")
print("\n文件大小:")
!ls -lh models_v2.0.0.zip

📦 打包模型文件...
  adding: models/v2.0.0/ (stored 0%)
  adding: models/v2.0.0/aggregator.pkl (deflated 73%)
  adding: models/v2.0.0/catboost.pkl (deflated 78%)
  adding: models/v2.0.0/ensemble.pkl (deflated 73%)
  adding: models/v2.0.0/lightgbm.pkl (deflated 65%)
  adding: models/v2.0.0/feature_list.json (deflated 59%)
  adding: models/v2.0.0/label_encoder.pkl (deflated 16%)
  adding: models/v2.0.0/manifest.json (deflated 54%)
  adding: models/v2.0.0/classification_report.txt (deflated 55%)
  adding: models/v2.0.0/xgboost.pkl (deflated 76%)

✅ 打包完成！

文件大小:
-rw-r--r-- 1 root root 7.4M Feb 14 14:25 models_v2.0.0.zip


In [11]:
# 10. 下载模型文件到本地
from google.colab import files

print("⬇️ 开始下载模型文件...")
print("（下载完成后解压到本地项目的 models/ 目录）\n")

files.download('models_v2.0.0.zip')

print("\n✅ 下载完成！")
print("\n📝 后续步骤:")
print("1. 解压 models_v2.0.0.zip")
print("2. 将 models/v2.0.0/ 目录复制到本地项目")
print("3. 运行推理: python scripts/infer.py data/xxx.xlsx --model-version v2.0.0")

⬇️ 开始下载模型文件...
（下载完成后解压到本地项目的 models/ 目录）



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 下载完成！

📝 后续步骤:
1. 解压 models_v2.0.0.zip
2. 将 models/v2.0.0/ 目录复制到本地项目
3. 运行推理: python scripts/infer.py data/xxx.xlsx --model-version v2.0.0


In [12]:
# 11. 测试推理（可选）
import glob

excel_files = glob.glob('data/**/*.xlsx', recursive=True)
if excel_files:
    test_file = excel_files[0]
    print(f"🧪 测试推理: {test_file}")

    !python scripts/infer.py "{test_file}" --model-version v2.0.0 --job-id colab_test

    print("\n✅ 推理完成！")
    print("\n查看结果:")
    !ls -la data/outputs/colab_test/
else:
    print("❌ 未找到测试数据文件")

🧪 测试推理: data/原始告警信息样例数据集V1.2.xlsx
2026-02-14 14:25:20,786 INFO === 推理任务开始: colab_test ===
2026-02-14 14:25:20,786 INFO [1/6] 数据导入标准化 ...
2026-02-14 14:25:21,973 INFO 读取 原始告警信息样例数据集V1.2.xlsx 成功，形状: (11000, 8)
2026-02-14 14:25:22,064 INFO 已保存标准化数据: /content/WuTong/data/staging/原始告警信息样例数据集V1.2.parquet  (11000 条)
2026-02-14 14:25:22,094 INFO 加载数据: 11000 条, 8 列
2026-02-14 14:25:22,094 INFO [2/6] 特征提取 ...
2026-02-14 14:25:22,094 INFO 开始特征提取，共 11000 条记录 ...
2026-02-14 14:25:22,811 INFO   特征提取进度: 2000 / 11000
2026-02-14 14:25:23,565 INFO   特征提取进度: 4000 / 11000
2026-02-14 14:25:23,995 INFO   特征提取进度: 6000 / 11000
2026-02-14 14:25:24,432 INFO   特征提取进度: 8000 / 11000
2026-02-14 14:25:24,939 INFO   特征提取进度: 10000 / 11000
2026-02-14 14:25:25,237 INFO 特征提取完成，新增 26 个特征列
2026-02-14 14:25:25,242 INFO [3/6] 告警降噪 ...
2026-02-14 14:25:25,243 INFO AlertAggregator 已加载: /content/WuTong/models/v2.0.0/aggregator.pkl
2026-02-14 14:25:25,243 INFO 开始告警聚合（推理模式），共 11000 条 ...
2026-02-14 14:25:28,257 INFO 推理聚类数: 29
202

---

## ✅ 训练完成！

### 📊 关键指标

- **模型架构**: XGBoost + CatBoost + LightGBM 三模型集成
- **预期准确率**: 98.5-99.5%
- **训练方式**: 5 折交叉验证
- **数据量**: 11,000 条告警数据
- **攻击类型**: 9 类（8 种攻击 + 正常访问）

### 📦 已生成文件

```
models/v2.0.0/
├── ensemble.pkl              # 集成模型（主要使用）
├── xgboost.pkl               # XGBoost 单模型
├── catboost.pkl              # CatBoost 单模型
├── lightgbm.pkl              # LightGBM 单模型
├── label_encoder.pkl         # 标签编码器
├── aggregator.pkl            # 告警聚合器
├── manifest.json             # 模型元数据
├── feature_list.json         # 特征列表
└── classification_report.txt # 详细分类报告
```

### 🚀 下一步

1. ✅ 下载 `models_v2.0.0.zip` 到本地
2. ✅ 解压到项目的 `models/` 目录
3. ✅ 在本地 CPU 上运行推理（完全兼容）
4. ✅ 集成到 API 服务或 Electron 前端

### ❓ 常见问题

**Q: Colab 训练的模型能在本地 CPU 运行吗？**  
A: 完全可以！模型训练时使用的是 CPU 指令，GPU 只是加速计算。下载后可以直接在任何 CPU 环境运行。

**Q: 准确率没达到预期怎么办？**  
A: 可以尝试：
- 增加交叉验证折数：`--cv-splits 10`
- 调整模型超参数
- 补充公开数据集

**Q: 模型文件太大怎么办？**  
A: 只保留 `ensemble.pkl`、`label_encoder.pkl`、`aggregator.pkl` 和配置文件即可，单模型文件可以删除。

---

**项目地址**: https://github.com/alltobebetter/WuTong
